# Tanque simple

En esta notebook estudio el comportamiento en el caso mas simple de la industria, el llenado de un tanque mediante una entrada y una salida.

## Balance de masa:
A=E-S+G-C

En este caso, al tratarse de un caso simple, la tasa de evaporacion de egua se desprecia, por lo que no hay ni consumo (C) ni generacion (G), resultando en :

A= E-S

Siendo la Entrada y Salida en caudales, puede re arreglarse la ecuacion para epresarla en aaltura, que es lo que se busca controlar: 

Area * dh/dt = Qin - Qout

## 1. IMPORTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import wandb
from typing import Dict, Any, List

# Imports del proyecto
from Environment.simulation_env import SimulationPIDEnv
from Simuladores.tanque_simple import TankSimulator
from Agentes.DQN.algorithm_DQN import DQNAgent

# Configuración de matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Imports completados")
print(f"PyTorch version: {torch.__version__}")
print(f"Device disponible: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 2. CONFIGURACIÓN DE HIPERPARÁMETROS

In [ ]:
# Configuración del experimento
config = {
    # Identificación
    'experiment_name': 'dqn_tank_control',
    'run_name': 'baseline_v1',
    
    # Ambiente
    'env': {
        'upper_range': 10.0,
        'lower_range': 0.0,
        'setpoint': 5.0,
        'dead_band': 0.2,
        'max_episode_steps': 200,
        'dt': 1.0
    },
    
    # Simulador de tanque
    'tank': {
        'area': 1.0,
        'cv': 0.1,
        'max_height': 10.0,
        'max_flow_in': 0.5,
        'dt': 1.0
    },
    
    # Agente DQN
    'agent': {
        'state_dim': 6,
        'action_dim': 7,
        'hidden_dims': (128, 128, 64),
        'lr': 0.001,
        'gamma': 0.99,
        'epsilon_start': 1.0,
        'epsilon_min': 0.01,
        'epsilon_decay': 0.995,
        'memory_size': 10000,
        'batch_size': 64,
        'target_update_freq': 100,
        'device': 'cpu'
    },
    
    # Entrenamiento
    'training': {
        'n_episodes': 100,
        'log_interval': 10,      # Logear cada N episodios
        'save_interval': 50,     # Guardar modelo cada N episodios
        'eval_interval': 25,     # Evaluar cada N episodios
        'n_eval_episodes': 5     # Episodios para evaluación
    }
}

print("✅ Configuración definida")
print(f"\nExperimento: {config['experiment_name']}")
print(f"Run: {config['run_name']}")
print(f"Episodios de entrenamiento: {config['training']['n_episodes']}")
print(f"Hidden layers: {config['agent']['hidden_dims']}")

## 3. INICIALIZAR WEIGHTS & BIASES

In [ ]:
# Inicializar W&B 
wandb.init(
    project=config['experiment_name'],
    name=config['run_name'],
    config=config,
    tags=['dqn', 'tank-control', 'pid-tuning']
)

print("✅ Weights & Biases inicializado")
print(f"Dashboard: {wandb.run.get_url()}")

## 4. Ambiente y Simulador de Tanque

In [ ]:
# Crear ambiente de simulación
env = SimulationPIDEnv(
    config=config['env'],
    control_mode='pid_tuning'
)

print("✅ Ambiente creado")

# Crear simulador de tanque
tank = TankSimulator(
    area=config['tank']['area'],
    cv=config['tank']['cv'],
    max_height=config['tank']['max_height'],
    max_flow_in=config['tank']['max_flow_in'],
    dt=config['tank']['dt']
)

print("✅ Simulador de tanque creado")


## 5. AGENTE DQN


In [ ]:

# Crear agente DQN
agent = DQNAgent(
    state_dim=config['agent']['state_dim'],
    action_dim=config['agent']['action_dim'],
    hidden_dims=config['agent']['hidden_dims'],
    lr=config['agent']['lr'],
    gamma=config['agent']['gamma'],
    epsilon_start=config['agent']['epsilon_start'],
    epsilon_min=config['agent']['epsilon_min'],
    epsilon_decay=config['agent']['epsilon_decay'],
    memory_size=config['agent']['memory_size'],
    batch_size=config['agent']['batch_size'],
    target_update_freq=config['agent']['target_update_freq'],
    device=config['agent']['device']
)

print("✅ Agente DQN creado")


In [ ]:
# Mostrar arquitectura de la red
print("\n📐 Arquitectura de la Red Q:")
print(agent.q_network)

print(f"\n📊 Parámetros totales: {sum(p.numel() for p in agent.q_network.parameters()):,}")

In [ ]:
# Mostrar estadísticas iniciales del agente
stats = agent.get_stats()

print("\n📈 Estadísticas del Agente:")
for key, value in stats.items():
    print(f"  {key}: {value}")